In [1]:
import numpy as np
import pandas as pd
import copy
import sklearn as sk
import torch
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression

import model
import verifier
import cscUpdater
import updater

import importlib as imp

import itertools
import time
import dill as pickle

from hummingbird.ml import convert

import warnings
warnings.filterwarnings('ignore')

import acsDataParallel

Importing plotly failed. Interactive plots will not work.


In [156]:
test_size = 0.2 #train-test split

acs_task = 'income' # options: employment, income, public_coverage, mobility, and travel_time.
acs_year = 2018 #must be >= 2014. Upper bound unknown.
acs_states = ['CA']
acs_horizon='1-Year' #1-Year or 5-Year
acs_survey='person' #'person' or 'household'

# for subsampling rows: can specify first and last of data to be pulled. currently pulling everything.
row_start = 0
row_end = 100000

# for subsampling columns. note: can only subsample consecutive columns with current implementation
col_start=0
col_end=-1

[train_x, train_y, test_x, test_y, demo_group_functions, demo_group_indicators, min_age, mid_age] = acsDataParallel.get_data(test_size, acs_task, acs_year, acs_states,acs_horizon=acs_horizon, acs_survey=acs_survey, row_start = row_start,row_end = row_end, col_start=col_start, col_end=col_end)

In [157]:
def g1(X):
    return ((X['WKHP'] == 40))

truth_series = g1(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf1 = sk.ensemble.RandomForestClassifier(n_estimators=100, max_depth=11)
clf1.fit(X_train,y_train)
clf1GPU = convert(clf1, 'pytorch')
clf1GPU.to('cuda')

def h1(x):
    return clf1GPU.predict(x)

def g2(X):
    return ((X['WKHP'] >= 20))

truth_series = g2(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf2 = sk.ensemble.RandomForestClassifier(n_estimators=100, max_depth=11)
clf2.fit(X_train,y_train)
clf2GPU = convert(clf1, 'pytorch')
clf2GPU.to('cuda')
def h2(x):
    return clf2GPU.predict(x)

def g3(X):
    return ((X['RAC1P'] != 1))

truth_series = g3(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf3 = sk.ensemble.RandomForestClassifier(n_estimators=200, max_depth=13)
clf3.fit(X_train,y_train)
clf3GPU = convert(clf1, 'pytorch')
clf3GPU.to('cuda')
def h3(x):
    return clf3GPU.predict(x)

def g4(X):
    return ((X['RAC1P'] != 0))

truth_series = g4(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf4 = sk.ensemble.RandomForestClassifier(n_estimators=200, max_depth=15)
clf4.fit(X_train,y_train)
clf4GPU = convert(clf1, 'pytorch')
clf4GPU.to('cuda')
def h4(x):
    return clf4GPU.predict(x)

def g5(X):
    return (X['AGEP'] >= 70)

truth_series = g5(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf5 = sk.ensemble.RandomForestClassifier(n_estimators=200, max_depth=10)
clf5.fit(X_train,y_train)
clf5GPU = convert(clf1, 'pytorch')
clf5GPU.to('cuda')
def h5(x):
    return clf5GPU.predict(x)

def g6(X):
    return (X['SCHL'] >= 16)

truth_series = g6(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf6 = sk.ensemble.RandomForestClassifier(n_estimators=200, max_depth=15)
clf6.fit(X_train,y_train)
clf6GPU = convert(clf1, 'pytorch')
clf6GPU.to('cuda')
def h6(x):
    return clf6GPU.predict(x)

def g7(X):
    return (X['AGEP'] >= 62)

truth_series = g7(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf7 = ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200,max_depth = 3,random_state=0)
clf7.fit(X_train,y_train)
clf7GPU = convert(clf1, 'pytorch')
clf7GPU.to('cuda')
def h7(x):
    return clf7GPU.predict(x)

def g8(X):
    return (X['SCHL'] >= 16)

truth_series = g8(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf8 = ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200,max_depth = 3,random_state=0)
clf8.fit(X_train,y_train)
clf8GPU = convert(clf1, 'pytorch')
clf8GPU.to('cuda')
def h8(x):
    return clf8GPU.predict(x)

def g9(X):
    return (X['SCHL'] <= 18)

truth_series = g8(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf9 = ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=300,max_depth = 8,random_state=0)
clf9.fit(X_train,y_train)
clf9GPU = convert(clf1, 'pytorch')
clf9GPU.to('cuda')

def h9(x):
    return clf9GPU.predict(x)

def g10(X):
    return (X['SCHL'] >= 1)

truth_series = g10(train_x)
X_train = train_x[truth_series]
y_train = train_y[truth_series]

clf10 = ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=300,max_depth = 8,random_state=0)
clf10.fit(X_train,y_train)
clf10GPU = convert(clf1, 'pytorch')
clf10GPU.to('cuda')
def h10(x):
    return clf10GPU.predict(x)

In [158]:
def verify_size(x, group):
# helper function that checks that the discovered group isn't too small to run on
    g_indices = group(x) == 1
    g_xs = x[g_indices]
    if len(g_xs) == 0:
        return False
    else:
        return True

In [159]:
initial_model = DecisionTreeClassifier(max_depth = 1, random_state=0)
initial_model.fit(train_x, train_y);

In [160]:
def gain_without_group(permutation, mod_without_group, predicate_list, group_list, train_x, train_y, test_x, test_y):
    for index in permutation:
        improvement_check = verifier.is_proposed_group_good_csc(mod_without_group, test_x, test_y, predicate_list[index],group_list[index])
        if improvement_check:
            cscUpdater.iterative_update(mod_without_group, predicate_list[index], group_list[index], train_x, train_y, test_x, test_y, 'g'+str(index))
    return mod_without_group.test_errors[-1][0]

In [230]:
group_list = [g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10]

In [255]:
def find_best_worst_perm(train_x, train_y, test_x, test_y, max_perms = None):
    start_time = time.time()
    permutations = list(itertools.permutations([1,5,6,9,10]))
    best_error = 100
    worst_error = 0
    best_permutation = None
    worst_permutation = None
    #set dummies
    g0 = None
    h0 = None

    #initialize lists
    group_list = [g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10]
    predicate_list = [h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10]
    counter = 0
    
    #initialize shapley value dictionary
    overall_shapley_values = {}
    for item in permutations[0]:
        overall_shapley_values[item] = 0
        
    if max_perms == None:
        permutations_set = permutations
    else:
        permutations_set = []
        random_selection = np.random.choice(len(permutations), size=max_perms, replace=False, p=None)
        for selection in random_selection:
            permutations_set.append(permutations[selection])
        
    for permutation in permutations_set:
        #round count
        if counter%10 == 0:
            print('Time for ' + str(counter) + ' permutations (seconds):', time.time() - start_time)

        #reinitialize model
        mod = model.PointerDecisionList(initial_model.predict, [])
        mod.test_errors.append(cscUpdater.measure_group_errors(mod, test_x, test_y))
        mod.train_errors.append(cscUpdater.measure_group_errors(mod, train_x, train_y))

        #position pointer
        position = 1

        #begin updates
        error_without_group_dict = {}

        for index in permutation:
            improvement_check = verifier.is_proposed_group_good_csc(mod, test_x, test_y, predicate_list[index],group_list[index])
            if improvement_check:
            # run the update & find Shapley contribution
                #save model prior to update
                file = open('pdl.pkl','wb')
                pickle.dump(mod,file)

                #make update
                cscUpdater.iterative_update(mod, predicate_list[index], group_list[index], train_x, train_y, test_x, test_y, 'g'+str(index))

                #error if you completed the computation without introducing the group
                file = open('pdl.pkl','rb')
                mod_without_group = pickle.load(file)
                if position != (len(permutation)-1):
                    error_without_group = gain_without_group(permutation[position:], mod_without_group, predicate_list, group_list, train_x, train_y, test_x, test_y)
                else:
                    error_without_group = mod_without_group.test_errors[-1][0]
                error_without_group_dict[index] = error_without_group
            else:
                error_without_group_dict[index] = mod.test_errors[-1][0]
            #position pointer update
            position += 1

        #final permutation error
        total_error = mod.test_errors[-1][0]

        #add the contributions to the overall shapley array
        for key in overall_shapley_values:  
            overall_shapley_values[key] = overall_shapley_values[key] + (error_without_group_dict[key] - total_error)

        if total_error < best_error:
            best_error = total_error
            best_permutation = permutation
        if total_error > worst_error:
            worst_error = total_error
            worst_permutation = permutation
        counter += 1
    average_shapley_values = {}
    print(overall_shapley_values)
    for key in overall_shapley_values:   
        average_shapley_values[key] = overall_shapley_values[key]/len(permutations_set)
    finish_time = time.time()
    delta_time = finish_time - start_time
    print('Best error:',best_error)
    print('Best Order:', best_permutation)
    print('Worst error:',worst_error)
    print('Worst Order:', worst_permutation)
    print('Total time (seconds):',delta_time)
    print('Average Shapley Values: ', average_shapley_values)
    return average_shapley_values

In [253]:
print(average_shapley_values)

{4: 0.0001791666666666747, 5: 0.016854166666666698, 6: 0.0, 10: 0.0001791666666666747}


In [256]:
average_shapley_values = find_best_worst_perm(train_x, train_y, test_x, test_y)

Time for 0 permutations (seconds): 5.626678466796875e-05
Time for 10 permutations (seconds): 14.524834871292114
Time for 20 permutations (seconds): 29.654272317886353
Time for 30 permutations (seconds): 41.69549608230591
Time for 40 permutations (seconds): 54.43840742111206
Time for 50 permutations (seconds): 63.54894781112671
Time for 60 permutations (seconds): 78.08669400215149
Time for 70 permutations (seconds): 90.05894184112549
Time for 80 permutations (seconds): 104.32205200195312
Time for 90 permutations (seconds): 116.63544607162476
Time for 100 permutations (seconds): 125.48011660575867
Time for 110 permutations (seconds): 133.26459336280823
{1: 0.0, 5: 2.2346000000000035, 6: 0.15480000000000027, 9: 0.008400000000000407, 10: 0.16320000000000068}
Best error: 0.21414999999999995
Best Order: (1, 5, 6, 9, 10)
Worst error: 0.21414999999999995
Worst Order: (1, 5, 6, 9, 10)
Total time (seconds): 139.90841674804688
Average Shapley Values:  {1: 0.0, 5: 0.018621666666666696, 6: 0.001290

In [223]:
average_shapley_values = {
1: 0.00016435000000000278, 
2: 1.6699999999999383e-05, 
3: -2.099999999999991e-06,
4: -1.4999999999999346e-05, 
5: 0.008999250000000026, 
6: 0.00010545000000000515, 
7: 0.0008514500000000084,
8: 6.395000000000306e-05, 
9: -9.14999999999988e-06,
10: -8.599999999999608e-06
}

In [224]:
#100k instances, 10 groups, 1000 random permutations
Best error: 0.21214999999999995
Best Order: (1, 8, 2, 7, 6, 4, 3, 5, 10, 9)
Worst error: 0.21414999999999995
Worst Order: (1, 7, 10, 6, 2, 5, 4, 3, 8, 9)
Total time (seconds): 2538.2251105308533
Average Shapley Values:  {1: 0.00016435000000000278, 2: 1.6699999999999383e-05, 3: -2.099999999999991e-06, 4: -1.4999999999999346e-05, 5: 0.008999250000000026, 6: 0.00010545000000000515, 7: 0.0008514500000000084, 8: 6.395000000000306e-05, 9: -9.14999999999988e-06, 10: -8.599999999999608e-06}

SyntaxError: invalid syntax (647341076.py, line 2)

In [225]:
def sorted_dictionary_values(d,ascending=True):
    sorted_list = []
    sorted_tuples = sorted(d.items(), reverse = ascending, key=lambda x: x[1])
    for item in sorted_tuples:
        sorted_list.append(item[0])
    return sorted_list

In [237]:
expected_best_perm = sorted_dictionary_values(average_shapley_values,ascending=True)
expected_worst_perm = sorted_dictionary_values(average_shapley_values,ascending=False)

In [238]:
print(f'Expected best permutation: {expected_best_perm}')
print(f'Expected worst permutation: {expected_worst_perm}')

Expected best permutation: [5, 7, 1, 6, 8, 2, 3, 10, 9, 4]
Expected worst permutation: [4, 9, 10, 3, 2, 8, 6, 1, 7, 5]


In [239]:
mod = model.PointerDecisionList(initial_model.predict, [])
mod.test_errors.append(cscUpdater.measure_group_errors(mod, test_x, test_y))
mod.train_errors.append(cscUpdater.measure_group_errors(mod, train_x, train_y))
for index in expected_best_perm:
    cscUpdater.iterative_update(mod, predicate_list[index], group_list[index], train_x, train_y, test_x, test_y, 'g'+str(index))
print(f'Best expected error: {mod.test_errors[-1][0]}')

Best expected error: 0.21304999999999996


In [240]:
mod = model.PointerDecisionList(initial_model.predict, [])
mod.test_errors.append(cscUpdater.measure_group_errors(mod, test_x, test_y))
mod.train_errors.append(cscUpdater.measure_group_errors(mod, train_x, train_y))
for index in expected_worst_perm:
    cscUpdater.iterative_update(mod, predicate_list[index], group_list[index], train_x, train_y, test_x, test_y, 'g'+str(index))
print(f'Worst expected error: {mod.test_errors[-1][0]}')

Worst expected error: 0.21414999999999995


In [241]:
mod = model.PointerDecisionList(initial_model.predict, [])
mod.test_errors.append(cscUpdater.measure_group_errors(mod, test_x, test_y))
mod.train_errors.append(cscUpdater.measure_group_errors(mod, train_x, train_y))
for index in [1,2,3,4,5,6,7,8,9,10]:
    cscUpdater.iterative_update(mod, predicate_list[index], group_list[index], train_x, train_y, test_x, test_y, 'g'+str(index))
print(f'Worst expected error: {mod.test_errors[-1][0]}')

Worst expected error: 0.21184999999999998


Can we make this work w expectations?

In [7]:
selection = np.random.choice(len(permutations), size=100, replace=False, p=None)

In [168]:
a = {1: 1.2500000000000844e-05, 2: 1.4999999999998349e-06, 3: 1.1999999999998679e-05, 4: 7.999999999999119e-06, 5: 0.01038400000000002, 6: 9.100000000000552e-05, 7: 0.0012170000000000104, 8: 7.4500000000004e-05, 9: 4.999999999999449e-07, 10: -5.149999999999766e-05}

In [178]:
sorted_dictionary_values(a)

[5, 7, 6, 8, 1, 3, 4, 2, 9, 10]

In [177]:
def sorted_dictionary_values(d):
    sorted_list = []
    sorted_tuples = sorted(d.items(), reverse = True, key=lambda x: x[1])
    for item in sorted_tuples:
        sorted_list.append(item[0])
    return sorted_list